## Initial Notes

- Typical a single token prediction is trained using NextTokenPrediction (NTP) task / loss. Using Cross-Entropy (CE) loss.
- Speculative decoding will improve the task and is cheap for inference
- How multiple token loss will speed up training? Ignore speed-ups on inference


## Plan
1. Quickly scan some relevant papers. Found [Better & Faster Large Language Models via Multi-token Prediction](https://arxiv.org/pdf/2404.19737).
2. Recap the baseline model training in the code-base
3. Test model export and running inference
4. Read code & implement the auxilary heads and losses so the export is still valid
5. Finally, experiment with the losses and hyperparameters

Some thoughts:
- CE losses for (1-default), 2, 3, 4, k NTP as the first thing to try.
- Consistency loss between having access to -1, -2, -3 token (as standard in the masked attention) and not knowing the attention.

## Problems encountered

### Update the script for the smaller model
| model | dim | n_layers | n_heads | n_kv_heads | max context length | parameters | val loss | download
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 260K | 64 | 5 | 8 | 4 | 512 | 260K | 1.297 | [stories260K](https://huggingface.co/karpathy/tinyllamas/tree/main/stories260K)

### Update the code to have auxilary function 
Not optimized version simple adding `AUX_LOSS: int` function based on hyperparameter and I simply shifted the targets.

## Experiments notes
TBD and split in multiple cells

## Summary
TBD and split in multiple cells